In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from RNN import RNN
%load_ext autoreload
%autoreload 2

In [6]:
url = "https://github.com/swarj/MLProject/raw/main/apple_share_price.csv" #get the dataset
raw_df = pd.read_csv(url)
raw_df = raw_df.loc[::-1] #reverse order of rows to chronologically make it go from past --> present
raw_df = raw_df.assign(Time=range(len(raw_df)))
df = raw_df.reindex(['Time','Date', 'High', 'Low', 'Volume', 'Open', 'Close'], axis = 1) #make target variable last column, if we want close instead of open we can switch it
#we can try to predict the close value for the following day, need to add target close value  
df['TargetNextClose'] = df['Close'].shift(-1)
df.drop(df.tail(1).index,inplace=True) # drop last rows
df.style
print(df.head(10))
print(df.shape)
df.reset_index(inplace=True)
data_set = df.iloc[:, 0:8]
pd.set_option('display.max_columns', None)
data_set = data_set.drop(columns=["Date", "Time", "index"], axis=1)
print(data_set.head(10))

#(Jason) i added this for my forward method
# Parse the date column as a datetime object
df['Date'] = pd.to_datetime(df['Date'])
# Convert the date column to a float timestamp
df['Date'] = df['Date'].apply(lambda x: x.timestamp())
# Reset the index
df.reset_index(inplace=True)

# Select the first 8 columns as the dataset
# Print the first 10 rows of the updated dataframe
print(data_set)

      Time       Date   High    Low     Volume   Open  Close  TargetNextClose
1663     0   3-Jan-11  47.18  46.41  111280407  46.52  47.08            47.33
1662     1   4-Jan-11  47.50  46.88   77337001  47.49  47.33            47.71
1661     2   5-Jan-11  47.76  47.07   63879193  47.08  47.71            47.68
1660     3   6-Jan-11  47.89  47.56   75106626  47.82  47.68            48.02
1659     4   7-Jan-11  48.05  47.41   77982212  47.71  48.02            48.92
1658     5  10-Jan-11  49.03  48.17  112139482  48.40  48.92            48.81
1657     6  11-Jan-11  49.28  48.50  111019993  49.27  48.81            49.20
1656     7  12-Jan-11  49.20  48.86   75644310  49.04  49.20            49.38
1655     8  13-Jan-11  49.52  49.12   74536182  49.31  49.38            49.78
1654     9  14-Jan-11  49.78  49.21   77209748  49.41  49.78            48.66
(1663, 8)
    High    Low     Volume   Open  Close
0  47.18  46.41  111280407  46.52  47.08
1  47.50  46.88   77337001  47.49  47.33
2  47.76 

In [8]:
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range=(0,1))

In [3]:
# Create an instance 
rnn = RNN(input_size=2, output_size=1, hidden_size=3)
# Define some inputs
input_data = [[1, 0], [0, 1], [0.2550690257394217, 0.49543508709194095], [0, 0]]
output_data = [[2],[2],[2],[2]]
# Call the forward method and print the outputs
outputs = rnn.forward_pass(input_data)
print(outputs)

[array([[-0.26723022]]), array([[-0.4531496]]), array([[-0.31220132]]), array([[-0.27831296]])]


In [12]:
splitlimit = int(len(data_set)*0.8)
hidden_units = 64
time_steps = 30
input_features = 4
learning_rate = 0.01

Wxh = np.random.randn(hidden_units, input_features) * 0.01
Whh = np.random.randn(hidden_units, hidden_units) * 0.01
Why = np.random.randn(1, hidden_units) * 0.01
bh = np.zeros((hidden_units, 1))
by = np.zeros((1, 1))

def forward_propagation(X, Y, h_prev):
    cache = {}
    cache['h'] = {}
    cache['x'] = {}
    cache['h'][-1] = np.copy(h_prev)
    loss = 0
    
    for t in range(len(X)):
        cache['x'][t] = X[t].reshape(-1, 1)
        cache['h'][t] = np.tanh(np.dot(Wxh, cache['x'][t]) + np.dot(Whh, cache['h'][t-1]) + bh)
        Y_pred = np.dot(Why, cache['h'][t]) + by
        loss += (Y[t] - Y_pred)**2
        
    cache['loss'] = loss
    cache['Y_pred'] = Y_pred
    
    return cache

def backward_propagation(X, Y, cache):
    dWxh = np.zeros_like(Wxh)
    dWhh = np.zeros_like(Whh)
    dWhy = np.zeros_like(Why)
    dbh = np.zeros_like(bh)
    dby = np.zeros_like(by)
    dh_next = np.zeros_like(cache['h'][0])
    
    for t in reversed(range(len(X))):
        dy = -2 * (Y[t] - cache['Y_pred'])
        dWhy += np.dot(dy, cache['h'][t].T)
        dby += dy
        
        dh = np.dot(Why.T, dy) + dh_next
        dh_raw = (1 - cache['h'][t]**2) * dh
        dbh += dh_raw
        dWxh += np.dot(dh_raw, cache['x'][t].T)
        dWhh += np.dot(dh_raw, cache['h'][t-1].T)
        dh_next = np.dot(Whh.T, dh_raw)
        
    for dparam in [dWxh, dWhh, dWhy, dbh, dby]:
        np.clip(dparam, -5, 5, out=dparam)
        
    return dWxh, dWhh, dWhy, dbh, dby

h_prev = np.zeros((hidden_units, 1))

data = data_set
data = sc.fit_transform(data)
data = pd.DataFrame(data)
X, X_test = np.array(data.iloc[:, :-1])[:splitlimit], np.array(data.iloc[:, :-1])[splitlimit:]
Y, Y_test = np.array(data.iloc[:, -1]).reshape(-1, 1)[:splitlimit], np.array(data.iloc[:, -1]).reshape(-1, 1)[splitlimit:]

for i in range(100):
    cache = forward_propagation(X, Y, h_prev)
    dWxh, dWhh, dWhy, dbh, dby = backward_propagation(X, Y, cache)
    Wxh -= learning_rate * dWxh
    Whh -= learning_rate * dWhh
    Why -= learning_rate * dWhy
    bh -= learning_rate * dbh
    by -= learning_rate * dby
    h_prev = cache['h'][len(X) - 1]
    if i % 10 == 0:
        print('Loss:', cache['loss'])

h_prev = np.zeros((hidden_units, 1))
Y_preds = []
for t in range(len(X_test)):
    cache = forward_propagation([X_test[t]], [Y_test[t]], h_prev)
    Y_preds.append(cache['Y_pred'])
    h_prev = cache['h'][0]

# calculate the MSE
MSE = np.mean((Y_test - Y_preds)**2)
print('Mean Squared Error:', MSE)

Loss: [[207.43648777]]
Loss: [[30772.92795811]]
Loss: [[16975.54471547]]
Loss: [[16038.34742827]]
Loss: [[3764.0788765]]
Loss: [[3915.89313318]]
Loss: [[4599.58430993]]
Loss: [[8919.50540648]]
Loss: [[12129.93603831]]
Loss: [[19167.495002]]
Mean Squared Error: 25.41772164248237
